In [3]:
import pandas as pd
import re

In [4]:
def split(x, i):
    a=x.split()
    if i>0:
        return int(a[i])
    else: 
        return a[i]

def traitement_annotation(file):
    
    data = pd.read_csv(file, sep='\t', header=None)
    data.columns=['T', 'a_split', 'valeur']
    
    data['classe']=data["a_split"].apply(lambda x: split(x, 0))
    data['debut']=data["a_split"].apply(lambda x: split(x, 1))
    data['fin']=data["a_split"].apply(lambda x: split(x, 2))
    
    data=data.drop(columns=['a_split'])
    data=data.reindex(columns=['T', 'classe', 'valeur', 'debut', 'fin'])
    data=data.sort_values('debut').reset_index(drop=True)
    
    a_supprimer=[0]
    for i in range(1,data.shape[0]):
        if data.loc[i,'classe']==data.loc[i-1,'classe'] and data.loc[i-1,'fin']==data.loc[i,'debut']:
            data.loc[i,'valeur']=data.loc[i-1,'valeur']+data.loc[i,'valeur']
            data.loc[i,'debut']= data.loc[i-1,'debut']
            a_supprimer[-1]=1
            a_supprimer.append(0)
        else:
            a_supprimer.append(0)
    

    data['a_supprimer']=a_supprimer
    
    data=data[data.a_supprimer==0]
    data=data.drop(columns=['a_supprimer'])
    data=data.reset_index(drop=True)
    
    return data

def links_dataframe(df):
    relations= pd.DataFrame(columns=['classe_source','valeur_source', 'classe_dest', 'valeur_dest'])
    classe_source=[]
    valeur_source=[]
    classe_dest=[]
    valeur_dest=[]
    
    for i in range (df.shape[0]-1):
        #traitement des valeurs
        if df.loc[i+1,'classe']=='valeur' and df.loc[i+1,'debut']-df.loc[i,'fin']<100:
            classe_source.append(df.loc[i, 'classe'])
            valeur_source.append(df.loc[i,'valeur'])
            classe_dest.append('valeur')
            valeur_dest.append(df.loc[i+1,'valeur'])
            
        #pathologie
        if df.loc[i,'classe']=='pathologie':
            #tant qu'on ne tombe pas sur une autre pathologie, tout le discours est lié au mm pathologie
            j=i+1
            while df.loc[j,'classe']!='pathologie':
                #on ne prend pas les classes 'valeur' qui est déjà traité
                if df.loc[j,'classe']!='valeur':
                    classe_source.append(df.loc[i, 'classe'])
                    valeur_source.append(df.loc[i,'valeur'])
                    classe_dest.append(df.loc[j,'classe'])
                    valeur_dest.append(df.loc[j,'valeur'])
                #on s'arrête si c'est la fin du df
                if j==df.shape[0]-1:
                    break
                else:
                    j+=1
        
        #les symptomes
        if df.loc[i,'classe']=='sosy':
            #tant qu'on ne tombe pas sur une autre pathologie ou symptome, tout le discours est lié au mm symptome
            j=i+1
            while df.loc[j,'classe']!='pathologie' and df.loc[j,'classe']!='sosy':
                #on ne prend pas les classes 'valeur' qui est déjà traité
                if df.loc[j,'classe']!='valeur':
                    classe_source.append(df.loc[i, 'classe'])
                    valeur_source.append(df.loc[i,'valeur'])
                    classe_dest.append(df.loc[j,'classe'])
                    valeur_dest.append(df.loc[j,'valeur'])
                #on s'arrête si c'est la fin du df
                if j==df.shape[0]-1:
                    break
                else:
                    j+=1
                    
                
    relations['classe_source'],relations['valeur_source'], relations['classe_dest'], relations['valeur_dest']=classe_source,valeur_source,classe_dest, valeur_dest
    return relations

In [9]:
data = traitement_annotation("output/filepdf-839-cas.ann")
data

,T,classe,valeur,debut,fin
0,T53,sosy,masse palpébrale gauche,126,149
1,T28,anatomie,palpébrale gauche,132,149
2,T37,examen,examen ophtalmologique,183,205
3,T25,traitement,examen ophtalmologique,183,205
4,T27,anatomie,doigts,258,264
5,T54,sosy,tonus oculaire à 10 mmhg,277,301
6,T8,anatomie,oculaire,283,291
7,T36,valeur,10 mmhg,294,301
8,T20,pathologie,neurofibrome plexiforme important de la paupiè...,307,373
9,T45,anatomie,paupière supérieure gauche,347,373


In [10]:
links_dataframe(data)[:50]

,classe_source,valeur_source,classe_dest,valeur_dest
0,sosy,masse palpébrale gauche,anatomie,palpébrale gauche
1,sosy,masse palpébrale gauche,examen,examen ophtalmologique
2,sosy,masse palpébrale gauche,traitement,examen ophtalmologique
3,sosy,masse palpébrale gauche,anatomie,doigts
4,sosy,tonus oculaire à 10 mmhg,anatomie,oculaire
5,anatomie,oculaire,valeur,10 mmhg
6,pathologie,neurofibrome plexiforme important de la paupiè...,anatomie,paupière supérieure gauche
7,pathologie,neurofibrome plexiforme important de la paupiè...,examen,palpation
8,pathologie,neurofibrome plexiforme important de la paupiè...,sosy,nodules centimétriques
9,pathologie,exophtalmie,sosy,élévation du globe oculaire


In [11]:
def infos_texte(file):
    
    #Ouverture du fichier. On enlève les "\n".
    with open(file, 'r', encoding='utf-8') as txt:
        texte = txt.read()
    
    texte = re.sub("\n","", texte)
    texte = texte[:100]
    
    #Pour retrouver l'âge.
    regex_age_string = r'\s*([0-9]*?)\s*-*ans'
    regex_age_string = re.compile(regex_age_string, re.DOTALL)
    sentence = texte
    age_extract_string = re.search(regex_age_string, sentence)
    age = age_extract_string.group()
    age = age.lstrip()
    age = age.rstrip()
    
    age = age[:2]
    age = int(age)
    tranche=''
    if age<=6:
        tranche='0-6 ans'
    elif age>6 and age<=20:
        tranche='6-20 ans'
    elif age>20 and age<=40:
        tranche='20-40 ans'
    elif age>40 and age<=60:
        tranche='40-60 ans'
    elif age>60 and age<=80:
        tranche='60-80 ans'
    else:
        tranche='plus de 80 ans'
    
    #Pour retrouver le sexe.
    sexe=''
    masculin = ['Homme', 'homme', 'Monsieur', 'monsieur', 'Garçon', 'garçon', 'Mr ', 'mr ', 'Mr.']
    feminin = ['Femme', 'femme', 'Madame', 'madame', 'Fille', 'fille', 'Mme ', 'mme ', 'Mme.', 'Dame', 'dame', 'mme. ']
    
    for word in feminin:
        if word in texte:
            sexe = 'femme'
    
    for word2 in masculin:
        if word2 in texte:
            sexe = 'homme'
    
    return age, tranche, sexe

In [13]:
infos_texte('input/filepdf-87-cas.txt')

(71, '60-80 ans', 'homme')